In [4]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver


def get_url_flipkart(search_term):
    #Generate a url from search term
    template = 'https://www.flipkart.com/search?q={}&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off'
    search_term = search_term.replace(' ','+')
    
    # add term query to url
    url = template.format(search_term)
    
    # add page query placeholder
    url += '&sort=price_asc&page={}'
    
    return url


def extract_record_flipkart(item):
    #Extract and return data from a single record
    
    # description and url
    atag = item.find('div',attrs={'class':'_4rR01T'})
    description = atag.text.strip()
    url = 'https://www.flipkart.com' + item.find('a')['href']
    
    try:
        # price
        price_parent = item.find('div',attrs={'class':'_30jeq3 _1_WHN1'})
        price = price_parent.text
    except AttributeError:
        return
    
    try:
        # rank and rating
        rating = item.find('div',attrs={'class':'_3LWZlK'}).text
        review_count = item.find('span',{'class':'_2_R_DZ'}).text
    except AttributeError:
        rating = ''
        review_count = ''
        
    result = (description, price, rating, review_count, url)
    
    return result


def main_flipkart(search_term):
    '''Run main program routine'''
    # startup the webdriver
    driver = webdriver.Chrome()
    
    records = []
    url = get_url_flipkart(search_term)
    
    for page in range(1,21):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div',{'class':'_2kHMtA'})
        
        for item in results:
            record = extract_record_flipkart(item)
            if record:
                records.append(record)
                
    driver.close()
    
    # save the data to csv file
    with open('result.csv', 'a', newline = '', encoding = 'utf-8') as f:
        writer = csv.writer(f,delimiter = '|')
        writer.writerow(['Description', 'Price', 'Rating', 'ReviewCount', 'Url'])
        writer.writerows(records)

In [5]:
main_flipkart('monitor')